In [ ]:
# Attach location IDs for properties to addresses/tax codes that can be used to match them to city parcels

In [1]:
import pandas as pd
import geopandas as gpd

In [33]:
taxing = pd.read_csv('./../raw-data/assessments_eric_chronicle_2016-01-28.csv',
                      dtype={'TXTXY4': str, 'ABAUCD': str, 'ABABTX': str})

In [34]:
taxing = taxing.rename(columns={'ABADTX': 'tax_code',
                                'ABAUCD': 'parcel_id',
                                'TXTXY4': 'year',
                                'ABABTX': 'street_num',
                                'ABCHCD': 'street_dir',
                                'ABACCD': 'street_name',
                                'ABAECD': 'street_desc'
                                })

In [35]:
print taxing.columns
taxing.head()

Index([u'tax_code', u'parcel_id', u'year', u'TXRASD', u'TXRADS', u'street_num',
       u'ABEMCD', u'street_dir', u'street_name', u'street_desc', u'ABFXCD',
       u'ABEPCD', u'ABDFCD', u'ABBPCD', u' TXTAXA01 '],
      dtype='object')


,tax_code,parcel_id,year,TXRASD,TXRADS,street_num,ABEMCD,street_dir,street_name,street_desc,ABFXCD,ABEPCD,ABDFCD,ABBPCD,TXTAXA01
0,NaN,3930,2010,STREET,STREET MAINTENANCE,310,-314,S,TRACY,AVE,NaN,NaN,NaN,59715,$126.68
1,NaN,3930,2010,TREE,TREE MAINTENANCE,310,-314,S,TRACY,AVE,NaN,NaN,NaN,59715,$24.30
2,NaN,14090,2010,STREET,STREET MAINTENANCE,103,-111,S,4TH,AVE,NaN,NaN,NaN,59715,$42.23
3,NaN,14090,2010,STREET,STREET MAINTENANCE,103,-111,S,4TH,AVE,NaN,NaN,NaN,59715,$42.23
4,NaN,14090,2010,TREE,TREE MAINTENANCE,103,-111,S,4TH,AVE,NaN,NaN,NaN,59715,$8.10


In [ ]:
# def str_int_not_nan(value):
#     if pd.notnull(value):
#         return str(int(value))
#     else: return None

# taxing['street_num'] = taxing['ABABTX'].apply(str_int_not_nan)

# def str_not_nan(value):
#     if pd.isnull(value):
#         return ""
#     else:
#         return str(value)
# keys = ['street_num', 'ABCHCD', 'ABACCD', 'ABAECD', 'ABFXCD', 'ABDFCD']

# taxing['t_address'] = taxing[keys].apply(lambda x: ' '.join(x.map(str_not_nan)), axis=1)

In [36]:
taxing = taxing[taxing['year'] == '2016']

In [38]:
def str_not_nan(value):
    if pd.isnull(value):
        return ""
    else:
        return str(value)

addr_keys = ['street_num', 'street_dir', 'street_name', 'street_desc']
taxing['address'] = taxing[addr_keys].apply(lambda x: str.strip(' '.join(x.map(str_not_nan))), axis=1)

In [39]:
taxing.head()

,tax_code,parcel_id,year,TXRASD,TXRADS,street_num,ABEMCD,street_dir,street_name,street_desc,ABFXCD,ABEPCD,ABDFCD,ABBPCD,TXTAXA01,address
116,NaN,119540,2016,ACSD,ARTERIAL & COLLECTOR,2771,NaN,NaN,FEN,WAY,NaN,NaN,NaN,59718,$21.53,2771 FEN WAY
117,NaN,119540,2016,ACSD,ARTERIAL & COLLECTOR,2771,NaN,NaN,FEN,WAY,NaN,NaN,NaN,59718,$21.53,2771 FEN WAY
118,NaN,119540,2016,SILD 690,SILD: CATTAIL CREEK PH 2 & 3,2771,NaN,NaN,FEN,WAY,NaN,NaN,NaN,59718,$347.12,2771 FEN WAY
119,NaN,119540,2016,STREET,STREET MAINTENANCE,2771,NaN,NaN,FEN,WAY,NaN,NaN,NaN,59718,$142.13,2771 FEN WAY
120,NaN,119540,2016,STREET,STREET MAINTENANCE,2771,NaN,NaN,FEN,WAY,NaN,NaN,NaN,59718,$142.13,2771 FEN WAY


In [ ]:
# def to_string(value):
#     if pd.isnull(value):
#         return None
#     else:
#         return str(int(value))

# taxing['ABAUCD'] = taxing['ABAUCD'].apply(to_string)

In [ ]:
# taxing['TXTXY4'].unique()

In [40]:
df = taxing[['tax_code','parcel_id', 'address']]
df = df.drop_duplicates()

In [ ]:

# df.rename(columns={'ABADTX': 'tax_code', 'ABAUCD': 'parcel_id'}, inplace=True)

In [42]:
df.to_csv('./../geodata/code-to-parcel.csv')

In [43]:
df.head()

,tax_code,parcel_id,address
116,NaN,119540,2771 FEN WAY
147,NaN,133020,0 PARK
250,NaN,145800,26 E MENDENHALL ST
310,NaN,173140,0 OPEN SPACE
338,NaN,173150,0 OPEN SPACE


In [ ]:
%ls ./../geodata

In [44]:
gdf = gpd.read_file('./../geodata/parcels.shp')

In [45]:
gdf = gdf[['LANDUSE','TAXCODE','geometry']]

In [47]:
combined = gdf.merge(df, left_on='TAXCODE', right_on='tax_code', how='left')

In [49]:
combined = combined[['TAXCODE','parcel_id', 'address', 'LANDUSE','geometry']]

In [51]:
combined.head()

,TAXCODE,parcel_id,address,LANDUSE,geometry
0,RGH6557,35940,1902 S TRACY AVE,SFR,POLYGON ((-111.0371702908188 45.66023004384551...
1,RGH3077,48090,1412 S BOZEMAN AVE,SFR,POLYGON ((-111.0340748253462 45.66463963759285...
2,RGG67424,234090,1015 ROSA WAY,DTR,POLYGON ((-111.1077146074659 45.69007381260906...
3,RFH40712,141780,3107 AUGUSTA DR,SFR,POLYGON ((-111.0300231015683 45.71168717926448...
4,RGG31355,61560,3427 CASCADE ST,SFR,POLYGON ((-111.0840284781531 45.68188000765788...


In [53]:
combined.to_file('./../geodata/parcels-merged.shp')